In [4]:
import pandas as pd
import numpy as np
import geopandas as gpd

from shapely.geometry import Point
import matplotlib.pyplot as plt

try: 
    import urllib2 as urllib
except ImportError:
    import urllib.request as urllib
import json
import os

In [5]:
!curl https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD > Inspection_data.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  735k    0  735k    0     0   534k      0 --:--:--  0:00:01 --:--:--  534k
100 7342k    0 7342k    0     0  3102k      0 --:--:--  0:00:02 --:--:-- 3103k
100 14.0M    0 14.0M    0     0  4278k      0 --:--:--  0:00:03 --:--:-- 4279k
100 20.8M    0 20.8M    0     0  4885k      0 --:--:--  0:00:04 --:--:-- 4886k
100 27.7M    0 27.7M    0     0  5233k      0 --:--:--  0:00:05 --:--:-- 5626k
100 32.8M    0 32.8M    0     0  5252k      0 --:--:--  0:00:06 --:--:-- 6545k
100 39.1M    0 39.1M    0     0  5448k      0 --:--:--  0:00:07 --:--:-- 6559k
100 45.7M    0 45.7M    0     0  5598k      0 --:--:--  0:00:08 --:--:-- 6490k
100 52.5M    0 52.5M    0     0  5752k      0 --:--

In [7]:
rest_data = pd.read_csv('Inspection_data.csv')
print('Original shape:',rest_data.shape)

Original shape: (200255, 27)


In [6]:
rest_data['INSPECTION DATE'] = pd.to_datetime(rest_data['INSPECTION DATE'])
# rest_data['inspection_year'] = rest_data['INSPECTION DATE'].dt.year
rest_data = rest_data[rest_data['INSPECTION DATE'].dt.year!=1900]
#only study inspection after 2022
rest_data = rest_data[rest_data['INSPECTION DATE'].dt.year>=2022]
#fliter out record without score
rest_data = rest_data[~rest_data.SCORE.isnull()]
rest_data = rest_data[~rest_data.Latitude.isnull()]
rest_data = rest_data[~rest_data['VIOLATION CODE'].isnull()]
rest_data = rest_data[~rest_data['GRADE DATE'].isnull()]
rest_data = rest_data[rest_data.Latitude!=0]
#RECORD DATE: indicate the current day
#GRADE DATE == INSPECTION DATE
exclude_col = ['PHONE','BIN','STREET','Location Point1','RECORD DATE','Community Board','Council District','BUILDING','ZIPCODE','GRADE DATE']  #'BBL',
#nta > census tract
rest_data = rest_data.loc[:, ~rest_data.columns.isin(exclude_col)]
#convert to geopandas
LatList = rest_data.Latitude.tolist()
LonList = rest_data.Longitude.tolist()
rest_data['geometry'] = [Point(LonList[i],LatList[i]) for i in range(len(rest_data))]
rest_data = gpd.GeoDataFrame(rest_data,crs={'init':'epsg:4326'}) #use projection 
rest_data.to_crs(epsg=4326,inplace=True)

print('After Cleaning',rest_data.shape)
rest_data = rest_data.reset_index()
rest_data.head()
     

Original shape: (200255, 27)
After Cleaning (59135, 18)


C:\Users\mluci\anaconda3\envs\geo_env1\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,index,CAMIS,DBA,BORO,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,INSPECTION TYPE,Latitude,Longitude,Census Tract,BBL,NTA,geometry
0,72,50083955,CHEONG HEI RESTAURANT,Queens,Chinese,2023-03-22,Violations were cited in the following area(s).,09B,Thawing procedure improper.,Not Critical,20.0,Z,Cycle Inspection / Re-inspection,40.746163,-73.883564,26902.0,4.015060e+09,QN29,POINT (-73.88356 40.74616)
1,84,41380708,BURGER KING,Bronx,Hamburgers,2022-03-17,Violations were cited in the following area(s).,09B,Thawing procedures improper.,Not Critical,9.0,A,Cycle Inspection / Initial Inspection,40.819050,-73.927091,6300.0,2.023470e+09,BX63,POINT (-73.92709 40.81905)
2,95,50059837,GOGI 37,Manhattan,Korean,2022-12-07,Violations were cited in the following area(s).,02B,Hot TCS food item not held at or above 140 °F.,Critical,16.0,B,Cycle Inspection / Re-inspection,40.753507,-73.990728,10900.0,1.007860e+09,MN17,POINT (-73.99073 40.75351)
3,109,50047617,TOAST & ROAST,Queens,American,2022-10-29,Violations were cited in the following area(s).,02B,Hot TCS food item not held at or above 140 °F.,Critical,12.0,A,Cycle Inspection / Initial Inspection,40.761527,-73.924429,6100.0,4.006140e+09,QN70,POINT (-73.92443 40.76153)
4,145,41710730,NORTH SHORE PIZZERIA & RESTAURANT,Staten Island,Pizza,2022-12-08,Violations were cited in the following area(s).,09B,Thawing procedure improper.,Not Critical,4.0,A,Cycle Inspection / Initial Inspection,40.625397,-74.135271,20100.0,5.003890e+09,SI07,POINT (-74.13527 40.62540)
